In [1]:
import sys
import tensorflow as tf
from numpy.random import RandomState

In [2]:
batch_size = 8

In [3]:
w1 = tf.Variable(tf.random_normal((2, 3), stddev=1, seed=1))
w2 = tf.Variable(tf.random_normal((3, 1), stddev=1, seed=1))

In [4]:
# 在 shape 的一个维度上使用 None 可以方便使用不同的 batch_size。
# 在训练时需要把数据分成比较小的 batch，但是在测试时，可以一次性使用全部的数据。
# 当数据集比较小时方便测试，但数据集比较大时，将大量数据放入一个 batch 可能会导致内存溢出。
x = tf.placeholder(tf.float32, shape=(None, 2), name="x-input")
y_ = tf.placeholder(tf.float32, shape=(None, 1), name="y-input")

In [5]:
y =  tf.matmul(tf.matmul(x, w1), w2)

In [6]:
y = tf.sigmoid(y)
# y: probability of positive samples
# 1-y: probability of negative samples

**什么是交叉熵？**

- 交叉熵（cross entropy）刻画了两个概率分布之间的距离，是分类问题中使用较多的一种损失函数。
- 公式：$H(p,q)=-\sum\limits_{x}p(x)\log{q(x)}$

**三分类问题的例子**

- 某个样例的正确答案是：(1,0,0)

- 某模型的预测答案是：(0.5,0.4,0.1)
  - 交叉熵 $H((1,0,0),(0.5,0.4,0.1))=-(1*\log{0.5}+0*\log{0.4}+0*\log{0.1})=\log{2}\approx0.3$
- 另一个模型的预测是：(0.8,0.1,0.1)
  - 交叉熵 $H((1,0,0),(0.8,0.1,0.1))=-(1*\log{0.8}+0*\log{0.1}+0*\log{0.1})=\log{1.25}\approx0.1$

- 可以得出第二个预测答案要优于第一个。用矩阵来表达，则为：$q=\begin{bmatrix}0.5&0.2&0.3\\0.8&0.1&0.1\end{bmatrix}\\p=\begin{bmatrix}1&0&0\\1&0&0\end{bmatrix}\\H(p,q)=\begin{bmatrix}0.3\\0.1\end{bmatrix}$

In [7]:
# 其中 y_ 代表正确结果，y 代表预测结果。
# 通过 tf.clip_by_value 函数可以将一个张量的数值限制在一个范围，如 (1e-10, 1.0) 内，可以避免一些错误例如 log0。
# 对于每一个样例中的每一个类别交叉熵 p(x)logq(x) 的计算，得到了一个n*m的二维矩阵，n为 batch 中样例的数量，m 为分类的类别数。
# tf.reduce_mean 对这 n 行取平均得到一个 batch 的平均交叉熵。比如前面那个三分类的例子，batch_size = 2，平均后为 0.2
cross_entropy = -tf.reduce_mean(y_ * tf.log(tf.clip_by_value(y, 1e-10, 1.0)) + (1-y_) * tf.log(tf.clip_by_value((1-y), 1e-10, 1.0)))

In [8]:
learning_rate = 0.001

In [9]:
train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [10]:
# 通过随机数生成一个模拟数据集
rdm = RandomState(1)
dataset_size = 128
X = rdm.rand(dataset_size, 2)
# 定义规则来给出样本的标签
# x1+x2<1 positive
# otherwise negative
# 和 TensorFlow 游乐场中的表示法不大一样的地方是，在这里使用 0 表示负样本，1 表示正样本。
# 大部分解决分类问题的神经网络都会采用 0 和 1 的表示方法
Y = [[int(x1+x2 < 1)] for (x1, x2) in X]

In [11]:
with tf.Session() as sess:
    
    sess.run(tf.global_variables_initializer())
    
    print(sess.run(w1))
    print(sess.run(w2))
    
    STEPS = 5000
    for i in range(STEPS):
        start = (i * batch_size) % dataset_size
        end = min(start + batch_size, dataset_size)
        
        sess.run(train_step, feed_dict={x:X[start:end], y_:Y[start:end]})
        
        if i %1000 == 0:
            total_cross_entropy = sess.run(cross_entropy, feed_dict={x:X, y_:Y})
            print("After %d training step(s), cross entropy on all data is %g" % (i, total_cross_entropy))
    
    print(sess.run(w1))
    print(sess.run(w2))

[[-0.8113182   1.4845988   0.06532937]
 [-2.4427042   0.0992484   0.5912243 ]]
[[-0.8113182 ]
 [ 1.4845988 ]
 [ 0.06532937]]
After 0 training step(s), cross entropy on all data is 1.89805
After 1000 training step(s), cross entropy on all data is 0.655075
After 2000 training step(s), cross entropy on all data is 0.626172
After 3000 training step(s), cross entropy on all data is 0.615096
After 4000 training step(s), cross entropy on all data is 0.610309
[[ 0.02476976  0.56948674  1.6921943 ]
 [-2.1977348  -0.23668915  1.1143894 ]]
[[-0.4554469 ]
 [ 0.49110925]
 [-0.9811033 ]]
